In [5]:
from mlflow.tracking import MlflowClient

In [20]:
MLFLOW_TRACKING_URI= 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [21]:
# # To create new experiments
# client.create_experiment(name='my_cool_exp')
experiment=client.get_experiment_by_name('random-forest-hyperopt')
print(experiment)

<Experiment: artifact_location='/home/moses/Documents/mlops-zoomcamp/week2/model_artifacts/2', creation_time=1685345328160, experiment_id='2', last_update_time=1685345328160, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>


In [47]:
from mlflow.entities import ViewType

# To search for the number of runs in an experiment aswell as the info
runs=client.search_runs(experiment_ids=experiment.experiment_id, 
                        filter_string='', 
                        run_view_type=ViewType.ACTIVE_ONLY, 
                        max_results=5, 
                        order_by=['metrics.Rmse ASC'])

# To search for the number of runs in an experiment aswell as the info
best_run=client.search_runs(experiment_ids=experiment.experiment_id, 
                        filter_string='', 
                        run_view_type=ViewType.ACTIVE_ONLY, 
                        max_results=5, 
                        order_by=['metrics.Rmse ASC'])[0]

In [48]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['Rmse']:.4f}")

run id: 54365af48150420e82f5e43b57888cf4, rmse: 2.4498
run id: 2f85528a970e465c9dffd7711310a986, rmse: 2.4514
run id: 13893dd6aa9543d1a180824266fd06e6, rmse: 2.4516
run id: b4d2a64cb264444e97aa1b8ca2bff051, rmse: 2.4539
run id: 8c82dc87ceda4129a4185fc8690d7563, rmse: 2.4560


In [50]:
run_id = best_run.info.run_id
print(run_id)

54365af48150420e82f5e43b57888cf4


In [10]:
# Promoting the models to the registry

import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
run_id = 'runs:/3a2ad3288e994c65b5ca8fe7601cf126/model'
mod_uri='runs:/{run_id}/model'

mlflow.register_model(model_uri=run_id, name='nycxgb-regressor')

Registered model 'nycxgb-regressor' already exists. Creating a new version of this model...
2023/05/24 13:35:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nycxgb-regressor, version 3
Created version '3' of model 'nycxgb-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1684931749116, current_stage='None', description=None, last_updated_timestamp=1684931749116, name='nycxgb-regressor', run_id='3a2ad3288e994c65b5ca8fe7601cf126', run_link=None, source='/home/moses/Documents/mlops-zoomcamp/week2/mlruns/1/3a2ad3288e994c65b5ca8fe7601cf126/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [24]:
## TO tansition the models
latest_version=client.get_latest_versions(name='nycxgb-regressor')

for version in latest_version:
    print(f"Version:{version.version}, Stage: {version.current_stage}")

Version:2, Stage: Staging
Version:3, Stage: Production


In [19]:
client.transition_model_version_stage(name='nycxgb-regressor', 
                                      version=3, stage='staging', 
                                      archive_existing_versions=False)

<ModelVersion: aliases=[], creation_timestamp=1684931749116, current_stage='Staging', description=None, last_updated_timestamp=1684933222250, name='nycxgb-regressor', run_id='3a2ad3288e994c65b5ca8fe7601cf126', run_link=None, source='/home/moses/Documents/mlops-zoomcamp/week2/mlruns/1/3a2ad3288e994c65b5ca8fe7601cf126/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [21]:
## To annotate the model

client.update_model_version(name='nycxgb-regressor', 
                            version=3, 
                            description=f"The model version {version} was transmitted to stagging")

<ModelVersion: aliases=[], creation_timestamp=1684931749116, current_stage='Staging', description=('The model version <ModelVersion: aliases=[], '
 "creation_timestamp=1684931749116, current_stage='None', description=None, "
 "last_updated_timestamp=1684931749116, name='nycxgb-regressor', "
 "run_id='3a2ad3288e994c65b5ca8fe7601cf126', run_link=None, "
 "source='/home/moses/Documents/mlops-zoomcamp/week2/mlruns/1/3a2ad3288e994c65b5ca8fe7601cf126/artifacts/model', "
 "status='READY', status_message=None, tags={}, user_id=None, version=3> was "
 'transmitted to stagging'), last_updated_timestamp=1684933529187, name='nycxgb-regressor', run_id='3a2ad3288e994c65b5ca8fe7601cf126', run_link=None, source='/home/moses/Documents/mlops-zoomcamp/week2/mlruns/1/3a2ad3288e994c65b5ca8fe7601cf126/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [22]:
# To promote a model to production
client.transition_model_version_stage(name='nycxgb-regressor', 
                                      version=3, 
                                      stage='production', 
                                      archive_existing_versions=False)


<ModelVersion: aliases=[], creation_timestamp=1684931749116, current_stage='Production', description=('The model version <ModelVersion: aliases=[], '
 "creation_timestamp=1684931749116, current_stage='None', description=None, "
 "last_updated_timestamp=1684931749116, name='nycxgb-regressor', "
 "run_id='3a2ad3288e994c65b5ca8fe7601cf126', run_link=None, "
 "source='/home/moses/Documents/mlops-zoomcamp/week2/mlruns/1/3a2ad3288e994c65b5ca8fe7601cf126/artifacts/model', "
 "status='READY', status_message=None, tags={}, user_id=None, version=3> was "
 'transmitted to stagging'), last_updated_timestamp=1684934031074, name='nycxgb-regressor', run_id='3a2ad3288e994c65b5ca8fe7601cf126', run_link=None, source='/home/moses/Documents/mlops-zoomcamp/week2/mlruns/1/3a2ad3288e994c65b5ca8fe7601cf126/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [23]:
## To annotate the model

client.update_model_version(name='nycxgb-regressor', 
                            version=3, 
                            description=f"The model version was transmitted to production")

<ModelVersion: aliases=[], creation_timestamp=1684931749116, current_stage='Production', description='The model version was transmitted to production', last_updated_timestamp=1684934118557, name='nycxgb-regressor', run_id='3a2ad3288e994c65b5ca8fe7601cf126', run_link=None, source='/home/moses/Documents/mlops-zoomcamp/week2/mlruns/1/3a2ad3288e994c65b5ca8fe7601cf126/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>